# Import fundmental ratio from FinViz

In [2]:
# For data manipulation
import pandas as pd

# To extract fundamental data
from bs4 import BeautifulSoup as bs
import requests

**Create a function to parse data from FinViz **

In [5]:
def fundamental_metric(soup, metric):
    return soup.find(text=metric).find_next(class_='snapshot-td2').text

In [11]:
def get_fundamental_data(df):
    for symbol in df.index:
        try:
            url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
            soup = bs(requests.get(url).content) 
            for m in df.columns:                
                df.loc[symbol,m] = fundamental_metric(soup,m)                
        except Exception as e:
            print (symbol, 'not found')
            print(e)
    return df

**List of Stocks and Ratios You are Interested In**

In [12]:

stock_list = ['AMZN','GOOG','PG','KO','IBM','DG','XOM','KO','PEP','MT','NL','GSB','LPL']

metric = ['P/B',
'P/E',
'Forward P/E',
'PEG',
'Debt/Eq',
'EPS (ttm)',
'Dividend %',
'ROE',
'ROI',
'EPS Q/Q',
'Insider Own'
]

**Initialize Pandas DataFrame to Store the Data

In [13]:
df = pd.DataFrame(index=stock_list,columns=metric)
df = get_fundamental_data(df)
df

AMZN not found
'NoneType' object has no attribute 'find_next'
GOOG not found
'NoneType' object has no attribute 'find_next'
PG not found
'NoneType' object has no attribute 'find_next'
KO not found
'NoneType' object has no attribute 'find_next'
IBM not found
'NoneType' object has no attribute 'find_next'
DG not found
'NoneType' object has no attribute 'find_next'
XOM not found
'NoneType' object has no attribute 'find_next'
KO not found
'NoneType' object has no attribute 'find_next'
PEP not found
'NoneType' object has no attribute 'find_next'
MT not found
'NoneType' object has no attribute 'find_next'
NL not found
'NoneType' object has no attribute 'find_next'
GSB not found
'NoneType' object has no attribute 'find_next'
LPL not found
'NoneType' object has no attribute 'find_next'


,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
AMZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GOOG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IBM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
